In [7]:
import torch
import transformers
from transformers import LlamaTokenizer, LlamaForCausalLM
import bitsandbytes as bnb
from transformers import BitsAndBytesConfig
import accelerate

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [9]:
model_id = "sarvamai/OpenHathi-7B-Hi-v0.1-Base"

In [10]:
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_use_double_quant=True,
    bnb_8bit_quant_type='nf8',  # Can be 'nf4' or 'fp4'
    bnb_8bit_compute_dtype=torch.bfloat16  # Adjust compute type if needed
)

In [11]:
tokenizer = LlamaTokenizer.from_pretrained(model_id)
model = LlamaForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map='auto',
    quantization_config=quantization_config,
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
def get_model_size(model):
    total_params = sum(p.numel() for p in model.parameters())
    total_size_bytes = total_params * 2
    total_size_gb = total_size_bytes / (1024 ** 3)
    print(f"Model size: {total_size_gb:.2f} GB")

get_model_size(model)

Model size: 12.80 GB


In [13]:
model.device

device(type='cuda', index=0)

In [14]:
def generate_response(prompt, max_length=40):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    generate_ids = model.generate(inputs.input_ids, max_length=max_length)
    output = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    response = output[0].split("\n---\n")
    del inputs
    torch.cuda.empty_cache()
    return f'Prompt: {response[0]}' + '\n' + f'Response: {response[1]}'

In [16]:
prompt = 'क्या सूर्य सूर्यमंडल का एकमात्र तारा है?'
output = generate_response(prompt)
print(output)

Prompt: क्या सूर्य सूर्यमंडल का एकमात्र तारा है?
Response: No, the Sun is not the only star in the Solar System. हमारे सौर मंडल में आठ ग्रह हैं, जिनमें पृथ्वी भी शामिल


In [17]:
# Write 'Has Ratan Tata ever played for Indian Cricket Team?' in Hindi
prompt = 'रतन टाटा कभी भारतीय क्रिकेट टीम के लिए खेले हैं?'
output = generate_response(prompt, max_length=40)
print(output)

Prompt: रतन टाटा कभी भारतीय क्रिकेट टीम के लिए खेले हैं?
Response: Yes, Tata has played for the Indian cricket team in the past. वह 1980 के दशक के


In [30]:
# Write 'Hindi is older than Sanskrit, yes or no?' in Hindi
prompt = 'हिंदी संस्कृत से पुरानी है, हाँ या नहीं?'

output = generate_response(prompt)
print(output)

Prompt: हिंदी संस्कृत से पुरानी है, हाँ या नहीं?
Response: Yes, Hindi is older than Sanskrit. हिंदी एक इंडो-आर्यन भाषा है जो भारत में


In [22]:
# Write 'What is the population of Delhi?' in Hindi
prompt = 'दिल्ली की जनसंख्या क्या है?'
output1 = generate_response(prompt)
print(output1)

# Write 'How many people live in Delhi?' in Hindi
prompt = 'दिल्ली में कितने लोग रहते हैं?'
output2 = generate_response(prompt)
print(output2)

# Write 'What is the count of the residents that live in Delhi' in Hindi
prompt = 'दिल्ली में रहने वाले निवासियों की गिनती क्या है?'
output3 = generate_response(prompt)
print(output3)

Prompt: दिल्ली की जनसंख्या क्या है?
Response: As of 2021, the population of Delhi is approximately 19.1 million people. यह भारत की राजधानी और सबसे अधिक
Prompt: दिल्ली में कितने लोग रहते हैं?
Response: As of 2021, the population of Delhi is approximately 18.9 million people. यह भारत की राजधानी और सबसे
Prompt: दिल्ली में रहने वाले निवासियों की गिनती क्या है?
Response: I don't have access to the latest population data for new delhi. हालाँकि, 201


In [31]:
# Write 'What is the name of the chairman of International Cricket Council?' in Hindi
prompt = 'अंतरराष्ट्रीय क्रिकेट परिषद के चेयरमैन का नाम क्या है?'
output1 = generate_response(prompt, max_length=60)
print(output1)

# Write 'THe news is that Jay Shah become the new chairman of International Cricket Council. Is it correct?' in Hindi
prompt = 'खबर यह है कि जय शाह अंतरराष्ट्रीय क्रिकेट परिषद के नए चेयरमैन बन गए हैं। क्या यह सही है? Yes or No?'
output2 = generate_response(prompt, max_length=60)
print(output2)

Prompt: अंतरराष्ट्रीय क्रिकेट परिषद के चेयरमैन का नाम क्या है?
A:
Response: The name of the Chairman of the International Cricket Council is Shashank Manohar.
Prompt: खबर यह है कि जय शाह अंतरराष्ट्रीय क्रिकेट परिषद के नए चेयरमैन बन गए हैं। क्या यह सही है? Yes or No?
जवाबः
Response: Yes, that Jay Shah has become the new chairman of the International Cricket Council.


In [10]:
# Write 'What is the real name of the Indian cricketer, Virat Kohli?' in Hindi
prompt = 'भारतीय क्रिकेटर, विराट कोहली का वास्तविक नाम क्या है?'
output1 = generate_response(prompt, max_length=60)
print(output1)

# Write 'I have heard that the Indian cricketer, Virat Kohli has a different name. What is it?' in Hindi
prompt = 'मैंने सुना है कि भारतीय क्रिकेटर, विराट कोहली का एक अलग नाम है। वह नाम क्या है?'
output2 = generate_response(prompt, max_length=60)
print(output2)

Prompt: भारतीय क्रिकेटर, विराट कोहली का असली नाम क्या है?
A:
Response: Virat Kohli's real name is Virat Anil Kumar Kohli.
Prompt: मैंने सुना है कि भारतीय क्रिकेटर, विराट कोहली का एक अलग नाम है। वह क्या है?
Response: हां, यह सही है। Virat Kohli is a famous Indian cricketer. उनका असली नाम वकार उल हसन खान है, लेकिन उन्हें अपने प्रशंसकों के बीच वकार या
